In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_csv(r"../Data/GPS/2022/CSV/939_cl_R.csv")
df[14495:14500]

,FID,GMT_Time,Latitude,Longitude,Altitude,Duration,Temperatur,DOP,Satellites,Cause_of_F,...,Lab_Alt,No,time,distance,dt,speed,FM3_R,FM3_G,FM3_T,RASTERVALU
14495,14495,2023-03-31 11:00:00,36.187219,-109.906385,1969.42,1.0,9.0,1.0,5.0,GPS Schedule,...,0.0,14834.0,2023-03-31 05:00:00,10.372575,10.0,0.636203,0.824114,0.0,0.0,0.272112
14496,14496,2023-03-31 11:10:00,36.187110,-109.906323,1972.50,2.0,9.0,1.2,5.0,GPS Schedule,...,0.0,14835.0,2023-03-31 05:10:00,13.387982,10.0,1.188028,0.698772,0.0,0.0,0.272375
14497,14497,2023-03-31 11:20:00,36.187076,-109.906449,1970.88,5.0,8.0,1.2,6.0,GPS Schedule,...,0.0,14836.0,2023-03-31 05:20:00,11.967402,10.0,1.267769,0.627418,0.0,0.0,0.264154
14498,14498,2023-03-31 11:30:00,36.187101,-109.906419,1982.27,2.0,8.5,1.2,6.0,GPS Schedule,...,0.0,14837.0,2023-03-31 05:30:00,3.913361,10.0,0.794038,0.824844,0.0,0.0,0.264154
14499,14499,2023-03-31 11:40:00,36.187098,-109.906411,1984.38,59.0,9.0,2.2,5.0,GPS Schedule,...,0.0,14838.0,2023-03-31 05:40:00,0.746180,10.0,0.232977,0.679392,0.0,0.0,0.264154


In [4]:
act_df_1 = pd.read_csv(r"../Data/ACC/2022/939/939_Act1.csv")
act_df_2 = pd.read_csv(r"../Data/ACC/2022/939/939_Act2.csv")
act_df_3 = pd.read_csv(r"../Data/ACC/2022/939/939_Act3.csv")
act_df_4 = pd.read_csv(r"../Data/ACC/2022/939/939_Act4.csv")
act_df_5 = pd.read_csv(r"../Data/ACC/2022/939/939_Act5.csv")
act_df_6 = pd.read_csv(r"../Data/ACC/2022/939/939_Act6.csv")
act_df_7 = pd.read_csv(r"../Data/ACC/2022/939/939_Act7.csv")

act_df = pd.concat([act_df_1, act_df_2, act_df_3, act_df_4, act_df_5, act_df_6, act_df_7])

act_df.tail()

,GMT Time,X,Y,Z,Temperature [C]
599208,6/11/2023 21:58,32,7,4,23.0
599209,6/11/2023 21:58,32,7,4,23.0
599210,6/11/2023 21:58,33,6,4,23.0
599211,6/11/2023 21:58,32,7,4,23.0
599212,6/11/2023 21:58,32,6,4,23.0


In [5]:
act_df['GMT Time'] = pd.to_datetime(act_df['GMT Time'], format = '%m/%d/%Y %H:%M')
df['GMT_Time'] = pd.to_datetime(df['GMT_Time'], format = '%Y-%m-%d %H:%M')
act_df.index = range(act_df.shape[0])

In [12]:
n = 0

# Initialize empty lists instead of arrays
X_int = []
Y_int = []
Z_int = []
ODBA_int = []
VeDBA_int = []

# Extract the required columns from act_df and df to avoid repeated lookups
act_gmt_time = act_df['GMT Time']
df_gmt_time = df['GMT_Time']

# Iterate through the rows of act_df
for i in tqdm(act_df.index):
    # Check if the current act_df time is within the interval
    if act_gmt_time[i] <= df_gmt_time[n] + pd.Timedelta(minutes=5):
        # Append values to the lists
        X_int.append(act_df.loc[i, 'X'])
        Y_int.append(act_df.loc[i, 'Y'])
        Z_int.append(act_df.loc[i, 'Z'])
        ODBA_int.append(abs(act_df.loc[i, 'X']) + abs(act_df.loc[i, 'Y']) + abs(act_df.loc[i, 'Z']))
        VeDBA_int.append((act_df.loc[i, 'X']**2 + act_df.loc[i, 'Y']**2 + act_df.loc[i, 'Z']**2)**0.5)
    else:
        # Compute the statistics and assign them to the df DataFrame
        df.loc[n, 'X_mean'] = np.mean(X_int)
        df.loc[n, 'X_std'] = np.std(X_int)
        df.loc[n, 'Y_mean'] = np.mean(Y_int)
        df.loc[n, 'Y_std'] = np.std(Y_int)
        df.loc[n, 'Z_mean'] = np.mean(Z_int)
        df.loc[n, 'Z_std'] = np.std(Z_int)
        df.loc[n, 'ODBA_mean'] = np.mean(ODBA_int)
        df.loc[n, 'ODBA_std'] = np.std(ODBA_int)
        df.loc[n, 'VeDBA_mean'] = np.mean(VeDBA_int)
        df.loc[n, 'VeDBA_std'] = np.std(VeDBA_int)

        # Reset the lists for the next interval
        X_int = []
        Y_int = []
        Z_int = []
        ODBA_int = []
        VeDBA_int = []

        # Increment n to move to the next interval in df
        n += 1


100%|██████████| 6890636/6890636 [16:06<00:00, 7131.95it/s]


In [6]:
df['GMT Time'] = df['GMT Time'].astype(str)
df['time'] = df['time'].astype(str)
df.head()
df.to_file(driver='ESRI Shapefile', filename=r"D:\PhD\P50\Data\Collar_Data\BlueGap\939\ArcMap\939_cl_act.shp")

In [67]:
a = [1,2,3]
b = [4,5,6]
a = np.array(a)
a = np.append(a,b)
a = np.insert(a,0,7)
a.mean()
a.std()

2.0